In [0]:
from pyspark.sql import SparkSession
import os
import glob
import shutil

# -----------------------------
# Inicializa Spark
# -----------------------------
spark = SparkSession.builder.appName("Bronze_API_Volume").getOrCreate()

# -----------------------------
# Recebe input_path via widget do Job/API
# -----------------------------
dbutils.widgets.text("input_path", "")
input_path = dbutils.widgets.get("input_path")

# -----------------------------
# Validação do input_path
# -----------------------------
if not input_path:
    print(" Nenhum input_path fornecido. Bronze não será executado.")
    df = None
elif not os.path.exists(input_path) or os.path.getsize(input_path) == 0:
    print(f" Arquivo não encontrado ou vazio: {input_path}. Bronze não será executado.")
    df = None
else:
    # -----------------------------
    # Detecta separador automaticamente (; ou ,)
    # -----------------------------
    sep_detected = None
    for sep_try in [";", ","]:
        df_try = spark.read.option("header", True).option("inferSchema", True).option("sep", sep_try).csv(input_path)
        if len(df_try.columns) > 1:
            sep_detected = sep_try
            df = df_try
            print(f" Separador detectado automaticamente: '{sep_try}'")
            break

    if not sep_detected:
        print(" Não foi possível detectar separador no CSV!")
        df = None

# -----------------------------
# Informações iniciais
# -----------------------------
if df is not None:
    df.show(5, truncate=False)
    print(f" Total de linhas: {df.count()}")


 Nenhum input_path fornecido. Bronze não será executado.


In [0]:
if df is not None and df.count() > 0:
    csv_dir = os.path.join(os.path.dirname(input_path), "bronze_csv_temp")
    final_csv = os.path.join(os.path.dirname(input_path), "bronze.csv")

    # Remove pasta temporária se existir
    shutil.rmtree(csv_dir, ignore_errors=True)

    # Salva CSV temporário
    df.coalesce(1).write.option("header", True).option("sep", ";").mode("overwrite").csv(csv_dir)

    # Move arquivo part-* para o destino final
    part_file = glob.glob(f"{csv_dir}/part-*")
    if not part_file:
        raise FileNotFoundError(f" Arquivo part-* não encontrado em {csv_dir}")
    shutil.move(part_file[0], final_csv)

    # Remove pasta temporária
    shutil.rmtree(csv_dir, ignore_errors=True)

    print(f" CSV legível gerado em: {final_csv}")
    final_csv_path = final_csv

else:
    final_csv_path = None
    print(" Nenhum dado para processar no Bronze.")


 Nenhum dado para processar no Bronze.


In [0]:
from pyspark.sql.functions import current_timestamp, from_utc_timestamp, date_format

if df is not None:
    df = df.withColumn("ingestion_time", from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"))
    df = df.withColumn("ingestion_time", date_format("ingestion_time", "yyyy/MM/dd-HH:mm:ss"))
    display(df)


In [0]:
import re

def normalizar_coluna(nome):
    nome = nome.lower()
    nome = re.sub(r'[^a-z0-9_]', '_', nome)
    nome = re.sub(r'_+', '_', nome)
    nome = nome.strip('_')
    return nome

if df is not None:
    novos_nomes = [normalizar_coluna(c) for c in df.columns]
    df = df.toDF(*novos_nomes)
    display(df)


In [0]:
if df is not None:
    df.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("saas_project.core.bronze_raw")


In [0]:
%sql
SELECT * FROM saas_project.core.bronze_raw LIMIT 10;


id,nome,idade,email,salario,cidade,ingestion_time
1,Pessoa 0,26,user0@exemplo.com,2036.32,null,2026/02/18-01:34:08
2,Pessoa 1,68,user1@exemplo.com,3705.09,Rio,2026/02/18-01:34:08
3,Pessoa 2,67,user2@exemplo.com,5693.16,São Paulo,2026/02/18-01:34:08
4,Pessoa 3,38,user3@exemplo.com,9376.51,Belo Horizonte,2026/02/18-01:34:08
5,Pessoa 4,53,user4@exemplo.com,6392.52,null,2026/02/18-01:34:08
6,Pessoa 5,49,user5@exemplo.com,7172.05,Rio,2026/02/18-01:34:08
7,Pessoa 6,null,user6@exemplo.com,3802.53,null,2026/02/18-01:34:08
8,Pessoa 7,47,user7@exemplo.com,4748.2,null,2026/02/18-01:34:08
9,Pessoa 8,27,user8@exemplo.com,4476.1,Belo Horizonte,2026/02/18-01:34:08
10,Pessoa 9,61,user9@exemplo.com,9242.02,null,2026/02/18-01:34:08


In [0]:
if df is not None:
    df_bronze = spark.table("saas_project.core.bronze_raw")
    df_bronze.show(5)
    print(f" Total de linhas no Bronze Delta: {df_bronze.count()}")
